<a href="https://colab.research.google.com/github/souhirbenamor/EPF/blob/main/DNN_flexible_june24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!pip install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


In [3]:
import os
from git.repo.base import Repo
Repo.clone_from("https://github.com/jeslago/epftoolbox", "epftoolbox")

<git.repo.base.Repo '/content/epftoolbox/.git'>

In [4]:
cd epftoolbox

/content/epftoolbox


In [5]:
EPF_1=pd.read_excel('/content/EPF_data_FUND.xlsx')
EPF_1.reset_index(inplace=True)
EPF_1["Date"] = pd.to_datetime(EPF_1["Date"])
EPF_1 = EPF_1.set_index('Date')
del EPF_1['index']
EPF_1.head()
print(EPF_1)

                     Price  Exogenous 1  Exogenous 2  Exogenous 3  \
Date                                                                
2011-01-09 00:00:00  18.29     40242.25      8280.00          0.0   
2011-01-09 01:00:00  16.04     39158.50      8412.75          0.0   
2011-01-09 02:00:00  14.60     38824.50      8502.25          0.0   
2011-01-09 03:00:00  14.95     38969.25      8668.50          0.0   
2011-01-09 04:00:00  14.50     39092.75      8849.25          0.0   
...                    ...          ...          ...          ...   
2017-01-08 19:00:00  56.70     49516.00      8053.50          0.0   
2017-01-08 20:00:00  52.44     47747.50      7456.50          0.0   
2017-01-08 21:00:00  51.86     47761.75      6902.50          0.0   
2017-01-08 22:00:00  52.26     46222.50      6395.00          0.0   
2017-01-08 23:00:00  50.87     43520.50      4633.50          0.0   

                     Exogenous 4  Exogenous 5  Exogenous 6  
Date                                     

In [6]:
EPF_1.to_csv('/content/EPF_data_FUND.csv')

In [7]:
dataset = 'EPF_data_FUND'

In [14]:
EPF_1

,Price,Exogenous 1,Exogenous 2,Exogenous 3,Exogenous 4,Exogenous 5,Exogenous 6
Date,,,,,,,
2011-01-09 00:00:00,18.29,40242.25,8280.00,0.0,21.30,55.17,7.27
2011-01-09 01:00:00,16.04,39158.50,8412.75,0.0,21.30,55.17,7.27
2011-01-09 02:00:00,14.60,38824.50,8502.25,0.0,21.30,55.17,7.27
2011-01-09 03:00:00,14.95,38969.25,8668.50,0.0,21.30,55.17,7.27
2011-01-09 04:00:00,14.50,39092.75,8849.25,0.0,21.30,55.17,7.27
...,...,...,...,...,...,...,...
2017-01-08 19:00:00,56.70,49516.00,8053.50,0.0,19.12,56.49,32.59
2017-01-08 20:00:00,52.44,47747.50,7456.50,0.0,19.12,56.49,32.59
2017-01-08 21:00:00,51.86,47761.75,6902.50,0.0,19.12,56.49,32.59


In [20]:
# Import necessary modules
from epftoolbox.data import read_data, DataScaler
import pandas as pd

# Read original data
df_train, df_test = read_data(path='.', dataset='EPF_data_FUND', begin_test_date='05/01/2015', end_test_date='08/01/2017')

# Extract values from original dataframes
Xtrain_orig = df_train.values
Xtest_orig = df_test.values

# Initialize the scaler
scaler = DataScaler('Norm')  # Replace 'Norm' with the desired scaling method

# Fit and transform the training data
Xtrain_scaled = scaler.fit_transform(Xtrain_orig)

# Transform the test data
Xtest_scaled = scaler.transform(Xtest_orig)

# Combine scaled training and test data into single DataFrames
df_combined_scaled = pd.concat([pd.DataFrame(Xtrain_scaled, index=df_train.index, columns=df_train.columns),
                               pd.DataFrame(Xtest_scaled, index=df_test.index, columns=df_test.columns)])

# Save combined scaled dataframe to CSV
df_combined_scaled.to_csv('EPF_exo_scaled.csv', index=True)

# Now, the combined scaled training and test data is saved into the CSV file 'EPF_data_FUND_combined_scaled.csv'


Test datasets: 2015-01-05 00:00:00 - 2017-01-08 23:00:00


Hyper

In [21]:
from epftoolbox.models import hyperparameter_optimizer

# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = 'EPF_exo_scaled'

# Number of years (a year is 364 days) in the test dataset.
years_test = 2

# Optional parameters for selecting the test dataset, if either of them is not provided,
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = '05/01/2015'
end_test_date = '08/01/2017'
# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = 4

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 1

# Number of iterations for hyperparameter optimization
max_evals = 1500

path_datasets_folder = "./datasets/"
path_hyperparameters_folder = "./experimental_files/"

In [ ]:
# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder,
                         path_hyperparameters_folder=path_hyperparameters_folder,
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset,
                         years_test=years_test, calibration_window=calibration_window,
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2015-01-05 00:00:00 - 2017-01-08 23:00:00




Tested 1/1500 iterations.
Best MAE - Validation Dataset
  MAE: 0.0 | sMAPE: 2.41 %

Best MAE - Test Dataset
  MAE: 0.0 | sMAPE: 4.06 %




Tested 2/1500 iterations.
Best MAE - Validation Dataset
  MAE: 0.0 | sMAPE: 2.41 %

Best MAE - Test Dataset
  MAE: 0.0 | sMAPE: 4.06 %
